# Train/Eval split (random stratified 80/20)
Loading the full dataset and creating train/eval pandas DataFrames.


In [1]:
import pandas as pd
from pathlib import Path

data_path = Path('Data/bank-additional-full.csv')
df = pd.read_csv(data_path, sep=';')
df.head()

###########################################
'''
Remove the duration comlumns from the models later
'''
############################################

'\nRemove the duration comlumns from the models later\n'

In [2]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(
    df, test_size=0.2, stratify=df["y"], random_state=42
)

print(f"Train rows: {len(train_df)}  Eval rows: {len(eval_df)}")
print('Train y counts: ', train_df['y'].value_counts())
print('Eval y counts: ', eval_df['y'].value_counts())


Train rows: 32950  Eval rows: 8238
Train y counts:  y
no     29238
yes     3712
Name: count, dtype: int64
Eval y counts:  y
no     7310
yes     928
Name: count, dtype: int64


In [3]:
# Split sanity checks
print('Train months:')
display(train_df['month'].value_counts())
print('Eval months:')
display(eval_df['month'].value_counts())


Train months:


month
may    11011
jul     5763
aug     4948
jun     4247
nov     3266
apr     2085
oct      587
sep      464
mar      436
dec      143
Name: count, dtype: int64

Eval months:


month
may    2758
jul    1411
aug    1230
jun    1071
nov     835
apr     547
oct     131
mar     110
sep     106
dec      39
Name: count, dtype: int64

## Model: predict high vs low response
Using logistic regression with one-hot encoding to classify clients as high-response (`yes`) or low-response (`no`).


In [4]:
# Prepare features/target
target_map = {'yes': 1, 'no': 0}
y_train = train_df['y'].map(target_map)
y_eval = eval_df['y'].map(target_map)

X_train = train_df.drop(columns=['y','duration'])
X_eval = eval_df.drop(columns=['y','duration'])

cat_cols = [c for c in X_train.columns if X_train[c].dtype == 'object']
num_cols = [c for c in X_train.columns if X_train[c].dtype != 'object']
print(f"Categorical cols: {len(cat_cols)}  Numeric cols: {len(num_cols)}")
for n in num_cols:
    print(n)
print("Categorical cols:")
for c in cat_cols:
    print(c)

Categorical cols: 10  Numeric cols: 9
age
campaign
pdays
previous
emp.var.rate
cons.price.idx
cons.conf.idx
euribor3m
nr.employed
Categorical cols:
job
marital
education
default
housing
loan
contact
month
day_of_week
poutcome


In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

preprocess = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('numeric', 'passthrough', num_cols),
    ],
    remainder='drop'
)

clf = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', LogisticRegression(max_iter=500, class_weight='balanced', n_jobs=-1)),
])
clf


,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('categorical', ...), ('numeric', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [6]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

clf.fit(X_train, y_train)

eval_pred = clf.predict(X_eval)
eval_proba = clf.predict_proba(X_eval)[:, 1]

print(f'Eval ROC-AUC: {roc_auc_score(y_eval, eval_proba):.3f}')
print('Classification report (eval):')
print(classification_report(y_eval, eval_pred, target_names=['low-response', 'high-response']))

cm = pd.DataFrame(
    confusion_matrix(y_eval, eval_pred),
    index=['true low', 'true high'],
    columns=['pred low', 'pred high'],
)
cm


Eval ROC-AUC: 0.798
Classification report (eval):
               precision    recall  f1-score   support

 low-response       0.95      0.85      0.90      7310
high-response       0.35      0.65      0.46       928

     accuracy                           0.82      8238
    macro avg       0.65      0.75      0.68      8238
 weighted avg       0.88      0.82      0.85      8238



,pred low,pred high
true low,6189,1121
true high,323,605


In [7]:
# Score the eval set to prioritize calls
scored_eval = eval_df.copy()
scored_eval['high_response_score'] = eval_proba
scored_eval[['high_response_score', 'y']].describe()


,high_response_score
count,8238.000000
mean,0.393512
std,0.205539
min,0.061011
25%,0.252607
50%,0.336717
75%,0.460040
max,0.976642


In [8]:
# Top 5 likely responders
scored_eval.sort_values('high_response_score', ascending=False)[['high_response_score', 'y', 'job', 'education', 'contact']]


# create a GUI that lets people input features and get a prediction from the model


,high_response_score,y,job,education,contact
40276,0.976642,yes,retired,high.school,cellular
40078,0.976085,yes,retired,basic.4y,cellular
40173,0.975110,no,technician,university.degree,cellular
40450,0.974869,yes,retired,unknown,cellular
39911,0.973826,yes,admin.,university.degree,cellular
...,...,...,...,...,...
5472,0.100164,no,blue-collar,basic.9y,telephone
6534,0.098272,no,blue-collar,basic.4y,telephone
5564,0.076969,no,admin.,university.degree,telephone
4114,0.069694,no,entrepreneur,university.degree,telephone


## Validation: cross-validation, lift, threshold
Extra checks to verify high/low-response predictions are meaningful. Run after fitting the model.


In [9]:
# Stratified CV on train set to check stability
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import precision_recall_curve, auc

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_proba = cross_val_predict(clf, X_train, y_train, cv=cv, method="predict_proba")[:, 1]
cv_pred = (cv_proba >= 0.5).astype(int)

print("Train CV ROC-AUC:", roc_auc_score(y_train, cv_proba))
print("Train CV classification report:")
print(classification_report(y_train, cv_pred, target_names=["low-response", "high-response"]))


Train CV ROC-AUC: 0.7870473745418102
Train CV classification report:
               precision    recall  f1-score   support

 low-response       0.95      0.85      0.89     29238
high-response       0.34      0.62      0.44      3712

     accuracy                           0.82     32950
    macro avg       0.64      0.73      0.67     32950
 weighted avg       0.88      0.82      0.84     32950



In [10]:
# Lift / decile analysis on eval set
import numpy as np

scored_eval = eval_df.copy()
scored_eval['high_response_score'] = eval_proba

scored_eval = scored_eval.sort_values("high_response_score", ascending=False)
scored_eval["decile"] = pd.qcut(scored_eval["high_response_score"], 10, labels=False, duplicates="drop")

lift = scored_eval.groupby("decile")["y"].apply(lambda s: (s == "yes").mean()).sort_index(ascending=False)
baseline_rate = (eval_df["y"] == "yes").mean()
lift_df = pd.DataFrame({
    "response_rate": lift,
    "lift_vs_baseline": lift / baseline_rate
})
print("Baseline eval response rate:", baseline_rate)
lift_df


Baseline eval response rate: 0.11264870114105366


,response_rate,lift_vs_baseline
decile,,
9,0.506068,4.492444
8,0.217233,1.928411
7,0.083738,0.743354
6,0.061968,0.550103
5,0.058252,0.517116
4,0.049757,0.441703
3,0.038882,0.345163
2,0.042476,0.377064
1,0.035194,0.312424


In [11]:
# Threshold tuning helper: choose cutoff to meet call capacity
call_capacity = 0.2  # fraction of eval set you can call; adjust as needed
cutoff = scored_eval["high_response_score"].quantile(1 - call_capacity)
print(f"Calling top {call_capacity*100:.0f}% => score cutoff {cutoff:.3f}")

subset = scored_eval[scored_eval["high_response_score"] >= cutoff]
precision = (subset["y"] == "yes").mean()
recall = (subset["y"] == "yes").sum() / (eval_df["y"] == "yes").sum()
print(f"Precision among called: {precision:.3f}")
print(f"Recall of all responders captured: {recall:.3f}")


Calling top 20% => score cutoff 0.511
Precision among called: 0.362
Recall of all responders captured: 0.642


## Model comparison: KNN
K-nearest neighbors with one-hot encoded categories and scaling for comparison to logistic regression.


In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Pipeline: one-hot categorical + passthrough numeric (scaled) for KNN
knn_preprocess = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('numeric', StandardScaler(with_mean=False), num_cols),
    ],
    remainder='drop'
)

knn_clf = Pipeline(steps=[
    ('preprocess', knn_preprocess),
    ('model', KNeighborsClassifier(n_neighbors=10, weights='distance', n_jobs=-1)),
])
knn_clf


,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('categorical', ...), ('numeric', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [13]:
# Fit/evaluate KNN on the same split
knn_clf.fit(X_train, y_train)
knn_pred = knn_clf.predict(X_eval)
knn_proba = knn_clf.predict_proba(X_eval)[:, 1]

print(f'KNN Eval ROC-AUC: {roc_auc_score(y_eval, knn_proba):.3f}')
print('Classification report (eval):')
print(classification_report(y_eval, knn_pred, target_names=['low-response', 'high-response']))

knn_cm = pd.DataFrame(
    confusion_matrix(y_eval, knn_pred),
    index=['true low', 'true high'],
    columns=['pred low', 'pred high'],
)
knn_cm


KNN Eval ROC-AUC: 0.754
Classification report (eval):
               precision    recall  f1-score   support

 low-response       0.92      0.97      0.94      7310
high-response       0.57      0.29      0.38       928

     accuracy                           0.89      8238
    macro avg       0.74      0.63      0.66      8238
 weighted avg       0.88      0.89      0.88      8238



,pred low,pred high
true low,7103,207
true high,659,269


## Model comparison: Decision Tree
Decision Tree with one-hot encoded categorical features for comparison.


In [14]:
from sklearn.tree import DecisionTreeClassifier

# One-hot encode categorical, passthrough numeric (no scaling needed for trees)
tree_preprocess = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('numeric', 'passthrough', num_cols),
    ],
    remainder='drop'
)

tree_clf = Pipeline(steps=[
    ('preprocess', tree_preprocess),
    ('model', DecisionTreeClassifier(
        max_depth=None,
        min_samples_leaf=20,
        class_weight='balanced',
        random_state=42,
    )),
])
tree_clf


,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('categorical', ...), ('numeric', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [15]:
# Fit/evaluate Decision Tree on the same split
tree_clf.fit(X_train, y_train)
tree_pred = tree_clf.predict(X_eval)
tree_proba = tree_clf.predict_proba(X_eval)[:, 1]

print(f'Decision Tree Eval ROC-AUC: {roc_auc_score(y_eval, tree_proba):.3f}')
print('Classification report (eval):')
print(classification_report(y_eval, tree_pred, target_names=['low-response', 'high-response']))

tree_cm = pd.DataFrame(
    confusion_matrix(y_eval, tree_pred),
    index=['true low', 'true high'],
    columns=['pred low', 'pred high'],
)
tree_cm


Decision Tree Eval ROC-AUC: 0.777
Classification report (eval):
               precision    recall  f1-score   support

 low-response       0.95      0.76      0.84      7310
high-response       0.26      0.67      0.37       928

     accuracy                           0.75      8238
    macro avg       0.60      0.71      0.61      8238
 weighted avg       0.87      0.75      0.79      8238



,pred low,pred high
true low,5525,1785
true high,304,624


## Model comparison: Naive Bayes
Gaussian Naive Bayes on one-hot encoded features for comparison (dense input).


In [16]:
from sklearn.naive_bayes import GaussianNB

# One-hot encode categorical as dense + passthrough numeric for Naive Bayes
nb_preprocess = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('numeric', 'passthrough', num_cols),
    ],
    remainder='drop'
)

nb_clf = Pipeline(steps=[
    ('preprocess', nb_preprocess),
    ('model', GaussianNB()),
])
nb_clf


,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('categorical', ...), ('numeric', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [17]:
# Fit/evaluate Naive Bayes on the same split
nb_clf.fit(X_train, y_train)
nb_pred = nb_clf.predict(X_eval)
nb_proba = nb_clf.predict_proba(X_eval)[:, 1]

print(f'Naive Bayes Eval ROC-AUC: {roc_auc_score(y_eval, nb_proba):.3f}')
print('Classification report (eval):')
print(classification_report(y_eval, nb_pred, target_names=['low-response', 'high-response']))

nb_cm = pd.DataFrame(
    confusion_matrix(y_eval, nb_pred),
    index=['true low', 'true high'],
    columns=['pred low', 'pred high'],
)
nb_cm


Naive Bayes Eval ROC-AUC: 0.776
Classification report (eval):
               precision    recall  f1-score   support

 low-response       0.93      0.91      0.92      7310
high-response       0.39      0.47      0.43       928

     accuracy                           0.86      8238
    macro avg       0.66      0.69      0.67      8238
 weighted avg       0.87      0.86      0.86      8238



,pred low,pred high
true low,6619,691
true high,491,437
